# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Bañados  ->  E. Bañados  |  ['E. Bañados']
F. Walter  ->  F. Walter  |  ['F. Walter']
Y. Guo  ->  Y. Guo  |  ['Y. Guo']
L. Acuna  ->  L. Acuna  |  ['L. Acuna']
R. Andrae  ->  R. Andrae  |  ['R. Andrae']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
Arxiv has 56 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2305.00998


extracting tarball to tmp_2305.00998...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Found documentclass in tmp_2305.00998/xqr30-paper1.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

extracting tarball to tmp_2305.01014... done.
Retrieving document from  https://arxiv.org/e-print/2305.01250


extracting tarball to tmp_2305.01250...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Found documentclass in tmp_2305.01250/aanda.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


Found 97 bibliographic references in tmp_2305.01250/aanda.bbl.
syntax error in line 86: '}' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2305.00998-b31b1b.svg)](https://arxiv.org/abs/arXiv:2305.00998) | **New quasar proximity zone size measurements at $z\sim 6$ using the  enlarged XQR-30 sample**  |
|| S. Satyavolu, et al. -- incl., <mark>E. Bañados</mark>, <mark>F. Walter</mark> |
|*Appeared on*| *2023-05-03*|
|*Comments*| *16 pages, 9 figures, Accepted in MNRAS*|
|**Abstract**| Proximity zones of high-redshift quasars are unique probes of their central supermassive black holes as well as the intergalactic medium in the last stages of reionization. We present 22 new measurements of proximity zones of quasars with redshifts between 5.8 and 6.6, using the enlarged XQR-30 sample of high-resolution, high-SNR quasar spectra. The quasars in our sample have UV magnitudes of $M_{1450}\sim -27$ and black hole masses of $10^9$$\unicode{x2013}$$10^{10}$ M$_\odot$. Our inferred proximity zone sizes are 2$\unicode{x2013}$7 physical Mpc, with a typical uncertainty of less than 0.5 physical Mpc, which, for the first time, also includes uncertainty in the quasar continuum. We find that the correlation between proximity zone sizes and the quasar redshift, luminosity, or black hole mass, indicates a large diversity of quasar lifetimes. Two of our proximity zone sizes are exceptionally small. The spectrum of one of these quasars, with $z=6.02$, displays, unusually for this redshift, damping wing absorption without any detectable metal lines, which could potentially originate from the IGM. The other quasar has a high-ionization absorber $\sim$0.5 pMpc from the edge of the proximity zone. This work increases the number of proximity zone measurements available in the last stages of cosmic reionization to 87. This data will lead to better constraints on quasar lifetimes and obscuration fractions at high redshift, which in turn will help probe the seed mass and formation redshift of supermassive black holes. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2305.01250-b31b1b.svg)](https://arxiv.org/abs/arXiv:2305.01250) | **Interior-atmosphere modelling to assess the observability of rocky  planets with JWST**  |
|| <mark>L. Acuna</mark>, M. Deleuil, O. Mousis |
|*Appeared on*| *2023-05-03*|
|*Comments*| *15 pages, 9 figures. Accepted for publication in A&A*|
|**Abstract**| Super-Earths present compositions dominated by refractory materials. However, there is a degeneracy in their interior structure between a planet with no atmosphere and a small Fe content, and a planet with a thin atmosphere and a higher core mass fraction. To break this degeneracy, atmospheric characterization observations are required. We present a self-consistent interior-atmosphere model to constrain the volatile mass fraction, surface pressure, and temperature of rocky planets with water and CO2 atmospheres. These parameters obtained in our analysis can then be used to predict observations in emission spectroscopy and photometry with JWST, which can determine the presence of an atmosphere, and if present, its composition. To obtain the bolometric emission and Bond albedo for an atmosphere in radiative-convective equilibrium, we present the k-uncorrelated approximation for fast computations within our retrieval on planetary mass, radius and host stellar abundances. For the generation of emission spectra, we use our k-correlated atmospheric model. An adaptive MCMC is used for an efficient sampling of the parameter space at low volatile mass fractions. We show how to use our modelling approach to predict observations with JWST for TRAPPIST-1 c and 55 Cancri e. TRAPPIST-1 c's most likely scenario is a bare surface, although the presence of an atmosphere cannot be ruled out. If the emission in the MIRI F1500 filter is 731 ppm or higher, there would be a water-rich atmosphere. For fluxes between 730 and 400 ppm, no atmosphere is present, while low emission fluxes (300 ppm) indicate a CO2-dominated atmosphere. In the case of 55 Cancri e, a combined spectrum with NIRCam and MIRI LRS may present high uncertainties at wavelengths between 3 and 3.7 $\mu$m. However, this does not affect the identification of H2O and CO2 because they do not present spectral features in this wavelength range. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2305.01014-b31b1b.svg)](https://arxiv.org/abs/arXiv:2305.01014) | **The Sloan Digital Sky Survey Reverberation Mapping Project: Key Results**  |
|| Y. Shen, et al. -- incl., <mark>Y. Guo</mark> |
|*Appeared on*| *2023-05-03*|
|*Comments*| *33 pages. Data products available at this ftp URL*|
|**Abstract**| We present the final data from the Sloan Digital Sky Survey Reverberation Mapping (SDSS-RM) project, a precursor to the SDSS-V Black Hole Mapper Reverberation Mapping program. This data set includes 11-year photometric and 7-year spectroscopic light curves for 849 broad-line quasars over a redshift range of 0.1<z<4.5 and a luminosity range of Lbol=1E44-47.5 erg/s, along with spectral and variability measurements. We report 23, 81, 125, and 110 reverberation mapping lags (relative to optical continuum variability) for broad Halpha, Hbeta, MgII and CIV using the SDSS-RM sample, spanning much of the luminosity and redshift ranges of the sample. Using 30 low-redshift RM AGNs with dynamical-modeling black hole masses, we derive a new estimate of the average virial factor of <log f>=0.62+-0.07 for the line dispersion measured from the RMS spectrum. The intrinsic scatter of individual virial factors is 0.31+-0.07 dex, indicating a factor of two systematic uncertainty in RM black hole masses. Our lag measurements reveal significant R-L relations for Hbeta and MgII at high redshift, consistent with the latest measurements based on heterogeneous samples. While we are unable to robustly constrain the slope of the R-L relation for CIV given the limited dynamical range in luminosity, we found substantially larger scatter in CIV lags at fixed L1350. Using the SDSS-RM lag sample, we derive improved single-epoch (SE) mass recipes for Hbeta, MgII and CIV, which are consistent with their respective RM masses as well as between the SE recipes from two different lines, over the luminosity range probed by our sample. The new Hbeta and MgII recipes are approximately unbiased estimators at given RM masses, but there are systematic biases in the CIV recipe. The intrinsic scatter of SE masses around RM masses is ~0.45 dex for Hbeta and MgII, increasing to ~0.58 dex for CIV. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2305.00998.md
    + _build/html/tmp_2305.00998/./dampingwingqso_paper_v2.png
    + _build/html/tmp_2305.00998/./fig2_11qsopanel.png
    + _build/html/tmp_2305.00998/./rphist_paper.png
exported in  _build/html/2305.01250.md
    + _build/html/tmp_2305.01250/./Figures/MRrel_comp_foreditor.png
    + _build/html/tmp_2305.01250/./Figures/OLR_vs_Tsurf_T1c.png
    + _build/html/tmp_2305.01250/./Figures/OLR_vs_Tsurf_55cnce.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcidauthorA}{0000-0001-5818-6838}$
$\newcommand{\orcidauthorB}{0000-0003-2895-6218}$
$\newcommand{\orcidauthorC}{0000-0001-5829-4716}$
$\newcommand{\orcidauthorD}{0000-0003-2344-263X}$
$\newcommand{\orcidauthorE}{0000-0001-8582-7012}$
$\newcommand{\orcidauthorF}{0000-0002-3324-4824}$
$\newcommand{\orcidauthorG}{0000-0002-5360-8103}$
$\newcommand{\orcidauthorH}{0000-0003-4793-7880}$
$\newcommand{\orcidauthorI}{0000-0002-4314-021X}$
$\newcommand{\orcidauthorJ}{0000-0003-3693-3091}$
$\newcommand{\orcidauthorK}{0000-0003-3310-0131}$
$\newcommand{\orcidauthorL}{0000-0002-6822-2254}$
$\newcommand{\orcidauthorM}{0000-0003-4793-7880}$
$\newcommand{\orcidauthorN}{0000-0001-8443-2393}$
$\newcommand{\orcidauthorO}{0000-0001-5211-1958}$
$\newcommand{\orcidauthorP}{0000-0003-4793-7880}$
$\newcommand{\orcidauthorQ}{0000-0003-4793-7880}$
$\newcommand{\orcidauthorR}{0000-0003-4793-7880}$
$\newcommand{\msun}{\mathrm{M}_\odot}$
$\newcommand{\lya}{Ly\alpha}$
$\newcommand{\lyb}{Ly\beta}$
$\newcommand{\HI}{H I}$
$\newcommand{\HII}{H II}$
$\newcommand{\HeI}{He I}$
$\newcommand{\HeII}{He II}$
$\newcommand{\HeIII}{He III}$
$\newcommand{\MgII}{Mg {\small II}}$
$\newcommand{\CII}{[C {\small II}]}$
$\newcommand{\CV}{C {\small V}}$
$\newcommand{\CIV}{C {\small IV}}$
$\newcommand{\SiIV}{Si {\small IV}}$
$\newcommand{\NV}{N\thinspace{V}}$
$\newcommand{\ud}{\mathrm{d}}$
$\newcommand{\nel}{n_\mathrm{e}}$
$\newcommand{\nH}{n_\mathrm{H}}$
$\newcommand{\nHe}{n_\mathrm{He}}$
$\newcommand{\nHI}{n_\mathrm{HI}}$
$\newcommand{\nHII}{n_\mathrm{HII}}$
$\newcommand{\nHeI}{n_\mathrm{HeI}}$
$\newcommand{\nHeII}{n_\mathrm{HeII}}$
$\newcommand{\nHeIII}{n_\mathrm{HeIII}}$
$\newcommand{\rp}{R_\mathrm{p}}$
$\newcommand{\rpc}{R_\mathrm{p,corr}}$
$\newcommand{\tq}{t_\mathrm{q}}$
$\newcommand{\gk}[1]{{\color{notecolor} [GK: #1]}}$
$\newcommand{\sr}[1]{{\color{color2} [SR: #1]}}$
$\newcommand{\change}[1]{{\color{purple} #1}}$
$\newcommand{\ace}[1]{{\color{green} [ACE: #1]}}$
$\newcommand{\theenumi}{\textbf{\Alph{enumi}.}}$
$\newcommand{\theenumii}{\textbf{\alph{enumii}.}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# New quasar proximity zone size measurements at $z\sim 6$ using the enlarged XQR-30 sample

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2305.00998-b31b1b.svg)](https://arxiv.org/abs/2305.00998)<mark>Appeared on: 2023-05-03</mark> -  _16 pages, 9 figures, Accepted in MNRAS_

</div>
<div id="authors">

S. Satyavolu, et al. -- incl., <mark>E. Bañados</mark>, <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** Proximity zones of high-redshift quasars are unique probes of their central supermassive black holes as well as the intergalactic medium in the last stages of reionization.  We present 22 new measurements of proximity zones of quasars with redshifts between 5.8 and 6.6, using the enlarged XQR-30 sample of high-resolution, high-SNR quasar spectra.  The quasars in our sample have UV magnitudes of $M_{1450}\sim -27$ and black hole masses of $10^9$ -- $10^{10}$ M $_\odot$ .  Our inferred proximity zone sizes are 2--7 physical Mpc, with a typical uncertainty of less than 0.5 physical Mpc, which, for the first time, also includes uncertainty in the quasar continuum.  We find that the correlation between proximity zone sizes and the quasar redshift, luminosity, or black hole mass,  indicates a large diversity of quasar lifetimes. Two of our proximity zone sizes are exceptionally small.  The spectrum of one of these quasars, with $z=6.02$ , displays, unusually for this redshift, damping wing absorption without any detectable metal lines, which could potentially originate from the IGM. The other quasar has a high-ionization absorber $\sim$ 0.5 pMpc from the edge of the proximity zone. This work increases the number of proximity zone measurements available in the last stages of cosmic reionization to 87.  This data will lead to better constraints on quasar lifetimes and obscuration fractions at high redshift, which in turn will help probe the seed mass and formation redshift of supermassive black holes.

</div>

<div id="div_fig1">

<img src="tmp_2305.00998/./dampingwingqso_paper_v2.png" alt="Fig6" width="100%"/>

**Figure 6. -** _Top panel_: Continuum-normalised spectrum of PSOJ158-14, for two continuum reconstruction methods, the log-PCA method \citep[]{2022ApJ...931...29C} and the covariance matrix method  ([Greig, et. al 2017](https://ui.adsabs.harvard.edu/abs/2017MNRAS.466.1814G)) , shown in blue and orange, respectively. Shaded regions show the $1\sigma$ spread around the median value.  (We use the log-PCA method for all quasars in this work.) _Middle panel_: A simulated spectrum showing an IGM damping wing at $z=6.14$  for a quasar with magnitude $-27$ and age 1 Myr. _Bottom panel_: The ionised hydrogen fraction along the same simulated sightline. This reveals the neutral hydrogen regions that create the damping wing seen in the middle panel. At redshift 6.14, only one of 500 sightlines in our simulation shows this feature. (*fig:dwingprops*)

</div>
<div id="div_fig2">

<img src="tmp_2305.00998/./fig2_11qsopanel.png" alt="Fig8" width="100%"/>

**Figure 8. -** Proximity zones of the quasars in our sample.  The normalised flux obtained by dividing measured flux by continuum, is shown in black. Red curves show the smoothed spectra with shaded region showing the 1$\sigma$ uncertainty in the continuum. Black solid and dotted lines show the quasar location and the extent of proximity zones, respectively. The blue shaded regions show the 1$\sigma$ uncertainty on proximity zone sizes due to continuum uncertainties.  Green shaded regions show redshift errors as the uncertainty on the location of the expected $\lya$ emission of the quasar.   (*fig:fullsample*)

</div>
<div id="div_fig3">

<img src="tmp_2305.00998/./rphist_paper.png" alt="Fig1" width="100%"/>

**Figure 1. -** Distribution of proximity zone sizes reported in this work.  The blue histogram shows the distribution of all previously available proximity zone sizes  ([ and Carilli 2010](https://ui.adsabs.harvard.edu/abs/2010ApJ...714..834C), [Eilers, et. al 2017](https://ui.adsabs.harvard.edu/abs/2017ApJ...840...24E), [ and Mazzucchelli 2017](https://ui.adsabs.harvard.edu/abs/2017ApJ...849...91M), [ and Reed 2017](https://ui.adsabs.harvard.edu/abs/2017MNRAS.468.4702R), [ and Bañados 2018](https://ui.adsabs.harvard.edu/abs/2018Natur.553..473B), [ and Eilers 2020](https://ui.adsabs.harvard.edu/abs/2020ApJ...900...37E), [ and Ishimoto 2020](https://ui.adsabs.harvard.edu/abs/2020ApJ...903...60I), [ and Bañados 2021](https://ui.adsabs.harvard.edu/abs/2021ApJ...909...80B)) , except those only available as values scaled to a fiducial luminosity, or that have been updated in this paper.  The yellow histogram shows the distribution of the 22 proximity zone sizes presented in this work. (*fig:rphist*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2305.00998"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Interior-atmosphere modelling to assess the observability of rocky planets with JWST

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2305.01250-b31b1b.svg)](https://arxiv.org/abs/2305.01250)<mark>Appeared on: 2023-05-03</mark> -  _15 pages, 9 figures. Accepted for publication in A&A_

</div>
<div id="authors">

L. Acuña, M. Deleuil, O. Mousis

</div>
<div id="abstract">

**Abstract:** Super-Earths present compositions dominated by refractory materials. However, there is a degeneracy in their interior structure between a planet with no atmosphere and a small Fe content, and a planet with a thin atmosphere and a higher core mass fraction. To break this degeneracy, atmospheric characterization observations are required. We present a self-consistent interior-atmosphere model to constrain the volatile mass fraction, surface pressure, and temperature of rocky planets with water and CO $_{2}$ atmospheres. These parameters obtained in our analysis can then be used to predict observations in emission spectroscopy and photometry with JWST, which can determine the presence of an atmosphere, and if present, its composition. We couple a 1D interior model with a supercritical water layer with an atmospheric model. To obtain the bolometric emission and Bond albedo for an atmosphere in radiative-convective equilibrium, we present the k-uncorrelated approximation for fast computations within our retrieval on planetary mass, radius and host stellar abundances. For the generation of emission spectra, we use our k-correlated atmospheric model. An adaptive Markov chain Monte Carlo (MCMC) is used for an efficient sampling of the parameter space at low volatile mass fractions. We show how to use our modelling approach to predict observations with JWST for TRAPPIST-1 c and 55 Cancri e, which have been proposed in Cycle 1. TRAPPIST-1 c's most likely scenario is a bare surface, although the presence of an atmosphere cannot be ruled out. If the emission in the MIRI F1500 filter is 731 ppm or higher, there would be a water-rich atmosphere. For fluxes between 730 and 400 ppm, no atmosphere is present, while low emission fluxes (300 ppm) indicate a CO $_{2}$ -dominated atmosphere. In the case of 55 Cancri e, a combined spectrum with NIRCam and MIRI LRS may present high uncertainties at wavelengths between 3 and 3.7 $\mu$ m. However, this does not affect the identification of H $_{2}$ O and CO $_{2}$ because they do not present spectral features in this wavelength range.

</div>

<div id="div_fig1">

<img src="tmp_2305.01250/./Figures/MRrel_comp_foreditor.png" alt="Fig7" width="100%"/>

**Figure 7. -** Comparison of the radius and interior-atmosphere boundary temperature between a k-correlated model and the k-uncorrelated approximation. Left panel: Mass-radius relationships for a planet with a water-dominated atmosphere orbiting a Sun-like star at $a_{d} = 0.05$ AU. Dashed lines indicate the total radius calculated by the k-uncorrelated version of MSEIRADTRAN, while the solid line corresponds to the interior radius, which comprises the core, mantle, and supercritical water (SW). Triangles and circles indicate the total radius and the interior radius obtained when the interior model is coupled with the atmospheric model of \cite{Pluriel19}, respectively. Right panel: Temperature at the 300 bar interface as a function of planetary mass. (*fig:MRdiag_MSEI*)

</div>
<div id="div_fig2">

<img src="tmp_2305.01250/./Figures/OLR_vs_Tsurf_T1c.png" alt="Fig5" width="100%"/>

**Figure 5. -** Outgoing longwave radiation (OLR) and Bond albedo as function of the surface temperature for a grey model, our k-correlated model, and the k-uncorrelated approximation, assuming the water-dominated case of TRAPPIST-1 c (see text). (*fig:OLR_T_T1c*)

</div>
<div id="div_fig3">

<img src="tmp_2305.01250/./Figures/OLR_vs_Tsurf_55cnce.png" alt="Fig6" width="100%"/>

**Figure 6. -** Outgoing longwave radiation (OLR) and Bond albedo as function of the surface temperature for a grey model, our k-correlated model, and the k-uncorrelated approximation, assuming the water-dominated case of 55 Cancri e (see text). (*fig:OLR_T_55cnce*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2305.01250"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

202  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
